In [239]:
import numpy as np
from collections import Counter
import porter as p
import re
from Weighter import Weighter
from Weighter1 import Weighter1

ImportError: cannot import name 'Weighter' from 'Weighter' (C:\Users\33658\Documents\GitHub\RITAL\tme3\Weighter.py)

- Données test (Docs / Requêtes)

In [148]:
docs=["the new home has been saled on top forecasts",
     "the home sales rise in july",
     "there is an increase in home sales in july",
     "july encounter a new home sales rise"]

reqs = ["top sales",
       "sales increase july",
       "new home"]

stopWords=["the","a","an","on","behind","under","there","in"]

index = {}
for i in range(len(docs)):
    index[i] = dict(Counter(map(p.stem, [word for word in (str.lower(docs[i])).split() if word not in stopWords])))           
            
indexInverse = {}
for numDoc, dico in index.items():
    for word, tf in dico.items():
        if(word not in indexInverse):
            indexInverse[word]= {}
        indexInverse[word][numDoc] = tf

- Pertinence des requêtes

In [149]:
pertiReq = {0:{0},      #doc1 pertinent pour req1
           1:{1,2},     #doc2 et 3 pertinent pour req2 (doc2 plus pertinent que doc3)
           2:{}}        #aucuns docs pertinents pour req3

In [150]:
if 1 in pertiReq[0]:
    print("hello")

## Exercice 1.

- modèle booléen

In [151]:
def booleen(query):
    req = list(np.unique(list(map(p.stem, query.split())))) 
    #représentation du résultat comme unensemble de documents
    res=set(index)
    for stem in req:
        res=res.intersection(indexInverse[stem])#On récupère l'intersection des documents contenant un mot de la requete
    return res

In [152]:
booleen(reqs[1])

{2}

- modele vectoriel

In [153]:
#modele vectoriel
def vect(query):
    nbDoc = 4
    req = list(np.unique(list(map(p.stem,query.split()))))
    res = [indexInverse[i] for i in req]
    doc = []
    doc_res = np.zeros(nbDoc)
    for w in res:
        doc.append(set(w.keys()))
    for w in doc:
        for d in w:
            doc_res[d]+=1
    return doc_res

#### Precision (capacité à ne retourner que des docs pertinents)

In [154]:
def preci(idQuery, modele = 0):
    if modele == 0:
        docs_retournés = booleen(reqs[idQuery])
    elif modele == 1:
        docs_retournés = vect(reqs[idQuery])
    pert = set(pertiReq[idQuery])     #docs jugés pertinents
    tp = docs_retournés.intersection(pert)  #True positifs (R & P)
    fp = docs_retournés.difference(preci)   #False positifs (R-P)
    if len(tp) == 0 & len(fp) == 0:
        return 0
    return len(tp)/(len(tp)+len(fp))     #mesure de précision

In [155]:
def preciRangk(rang,idQuery,score,jugement):
    """ score: dict() -> {idDoc: scoreDoc} dépend du modèle de poids adopté
        (trié par score décroissant)
        jugement : pertinence des requêtes"""
    k = rang
    res = 0
    for i in k:
        if score[i] in jugement[idQuery]:
            res+=1
    return res/k

#### Rappel (capacité à retourner tous les docs pertinents)

In [156]:
def rappel(idQuery, modele = 0):
    if modele == 0:
        docs_retournés = booleen(reqs[idQuery])
    elif modele == 1:
        docs_retournés = set(vect(reqs[idQuery]))
    pert = set(pertiReq[idQuery])      #docs jugés pertinents
    tp = docs_retournés.intersection(pert)    #True positifs (R & P)
    fn = pert.difference(docs_retournés)      #False negatifs (P-R)
    if len(tp) == 0 & len(fn) == 0:
        return 0
    return len(tp)/(len(tp)+len(fn))      #mesure de rappel

In [157]:
def rappelRangk(rang,idQuery,score,jugement):
    """ score: dict() -> {idDoc: scoreDoc} dépend du modèle de poids adopté
        (trié par score décroissant)
        jugement : pertinence des requêtes"""
    k = rang
    res = 0
    for i in k:
        if score[i] in jugement[idQuery]:
            res+=1
    return res/len(jugement[idQuery])

In [158]:
rappel(2,1)

0

In [159]:
def fMesure(idQuery, rang):
    r = rappel(idQuery)
    p = preci(idQuery)
    fact1 = 1+rang**2
    fact2 = (p*r)/(p*(rang**2)+r)
    return fact1*fact2

#### Mesure NCDG

In [32]:
def NDCG(idQuery):
    #TODO
    pass

## Exercice 2.

## 1. Chargement des requêtes et de leur pertinence

In [160]:
class Query():
    def __init__(self, idQuery):
        self.identifiant = int(idQuery)
        self.texte = ""
        self.docsPertinents = []
     
    #GETTERS
    def getIdentifiant(self):
        return self.identifiant
        
    def getTexte(self):
        return self.texte
        
    def getDocspertinents(self):
        return self.docsPertinents
    
    # SETTERS
    def setTexte(self, texte):
        self.texte = texte
        
    def setDocspertinents(self, docs):
        self.docsPertinents = docs
        
        
    # METHODES
    def addTexte(self, texte):
        self.texte += texte
        
    def addDocspertinents(self,doc):
        self.docsPertinents.append(doc)

In [161]:
class ParserQuery():
    
    def parseQRY(chemin):
        """ 
        Fonction permettant de parser les fichers QRY (requêtes avec leurs identifiants et leur texte)
        """
        file = open(chemin, 'r') 

        res = {}
        currentI = None
        currentBalise = None 
        currentQuery = None

        while True:
            #lis une seule ligne
            line = file.readline()

            #si ligne vide, fin du fichier
            if not line:
                break

            #récupère la ligne sous forme de mots
            words=line.split()

            #Si la ligne n'est pas vide
            if(len(words)>0):
                #Test si on est sur une balise et laquelle
                if(words[0]==".I"):

                    if(currentQuery != None):
                        #J'enregistre la requete courante avant d'en créer une autre
                        res[currentQuery.getIdentifiant()] = currentQuery 

                    del currentQuery
                    currentQuery = Query(words[1])# Création d'une requete avec son identifiant
                    currentI = words[1]
                    currentBalise = 'I' 

                elif(words[0]==".W"):
                    currentBalise='W' #J'indique que je suis danc une balise W
                elif(words[0][0]=='.'): 
                    currentBalise='unknown' #J'indique que je suis dans une balsie inconnue
                else: 
                    #On est dans le contenu d'une balise
                    if(currentBalise=='W'):
                        currentQuery.addTexte(line)#J'ajoute la ligne au texte de la requête

        #J'enregistre la requête courante avant de quitter
        res[currentQuery.getIdentifiant()] = currentQuery

        file.close()
        return res
    
    
    def parseREL(chemin, reqs):
        
        file = open(chemin, 'r') 
        
        while True:
            #lis une seule ligne
            line = file.readline()

            #si ligne vide, fin du fichier
            if not line:
                break
            words=line.split()
            
            if int(words[0][0]) == 0:
                docPertinent = int(re.split('^0*',words[1])[1])
                reqs[int(words[0][1])].addDocspertinents(docPertinent)
            else:
                docPertinent = int(re.split('^0*',words[1])[1])
                reqs[int(words[0])].addDocspertinents(docPertinent)
                     
        file.close()
        return reqs

In [162]:
req1 = ParserQuery.parseQRY('cacm.qry')
req2 = ParserQuery.parseREL('cacm.rel',req1)

In [163]:
req2[1].getDocspertinents()

[1410, 1572, 1605, 2020, 2358]

In [118]:
tel = '0100'
t = re.split('^0*',tel)
regex = re.compile('^[0-9]{2}\s[0-9]{4}')

In [119]:
t

['', '100']

In [88]:
int(tel.split()[0][0]) == 0

True

## 2. Métriques

In [209]:
class EvalMesure():
    
    def __init__(self,query):
        self.query = query
        
    def evalQuery(self,liste):
        pass
    
    #GETTERS
    def getQuery(self):
        return self.query

In [210]:
class Precision(EvalMesure):
    
    def __init__(self,query,k):
        
        super().__init__(query)
        self.rang = k
        
    def evalQuery(self, liste):
        """ liste: dict() -> {idDoc: scoreDoc} dépend du modèle de poids adopté
        (trié par score décroissant)
        jugement : pertinence des requêtes"""
        res = 0
        for i in self.rang:
            if score[i] in self.query.getDocspertinents():
                res+=1
        return res/self.rang

In [194]:
class Rappel(EvalMesure):
    
    def __init__(self,reqs,jugement,k):
        
        super().__init__(reqs,jugement)
        self.rang = k
        
    def evalQuery(self, liste):
        """ liste: dict() -> {idDoc: scoreDoc} dépend du modèle de poids adopté
        (trié par score décroissant)
        jugement : pertinence des requêtes"""
        rang = k
        res = 0
        for i in k:
            if score[i] in self.query.getDocspertinents():
                res+=1
        return res/len(self.query.getDocspertinents())
    

In [195]:
class AP(EvalMesure):
    """Avg precision"""
    
    def __init__(self,reqs,jugement,k):
        
        super().__init__(reqs,jugement)
        
    def evalQuery(self, liste):
        """ calcul de la preci moyenne"""
        nq = len(self.query.getDocspertinents()) #nb de doc pertinent pour q
        res = 0
        k=1 #initialisation du rang
        for i in score.keys():
            res+=self.preci(liste, k)
        return res/nq
    
    def preci(self, liste, rang):
        """ liste: dict() -> {idDoc: scoreDoc} dépend du modèle de poids adopté
        (trié par score décroissant)
        jugement : pertinence des requêtes"""
        res = 0
        for i in rang:
            if score[i] in self.query.getDocspertinents():
                res+=1
        return res/rang

## Test

In [213]:
r = ParserQuery.parseQRY('cacm.qry')
querys = ParserQuery.parseREL('cacm.rel',r)
query = querys[1]

In [214]:
k = 2 #rang
mesurePreci = Precision(query,k)
mesurePreci.getQuery().getDocspertinents()

[1410, 1572, 1605, 2020, 2358]

## 3. Plateforme d'évaluation